In [4]:

# This has been slightly modified for duolingo competition

import numpy as np
import tensorflow as tf
import time
import csv
import random
from sklearn.metrics import mean_squared_error, r2_score, f1_score, accuracy_score, log_loss
from sklearn import metrics
from math import sqrt
# flags
# tf.flags.DEFINE_float("epsilon", 0.1, "Epsilon value for Adam Optimizer.")
# tf.flags.DEFINE_float("l2_lambda", 0.3, "Lambda for l2 loss.")
# tf.flags.DEFINE_float("learning_rate", 0.1, "Learning rate")
# tf.flags.DEFINE_float("max_grad_norm", 20.0, "Clip gradients to this norm.")
# tf.flags.DEFINE_float("keep_prob", 0.6, "Keep probability for dropout")
# tf.flags.DEFINE_integer("hidden_layer_num", 1, "The number of hidden layers (Integer)")
# tf.flags.DEFINE_integer("hidden_size", 200, "The number of hidden nodes (Integer)")
# tf.flags.DEFINE_integer("evaluation_interval", 5, "Evaluate and print results every x epochs")
# tf.flags.DEFINE_integer("batch_size", 32, "Batch size for training.")
# tf.flags.DEFINE_integer("epochs", 150, "Number of epochs to train for.")
# tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
# tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
# tf.flags.DEFINE_boolean("save_test_logits", False, "Save test data logits on disk")
# tf.flags.DEFINE_string("train_data_path", 'data_fr_en/formatted_data_train.csv', "Path to the training dataset")
# tf.flags.DEFINE_string("test_data_path", 'data_fr_en/formatted_data_test.csv', "Path to the testing dataset")


# FLAGS = tf.flags.FLAGS
# FLAGS(sys.argv, known_only=True)
# remaining_args = FLAGS([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")])
# assert(remaining_args == [sys.argv[0]])
#FLAGS._parse_flags()
class FLAGS:
    kind = 'canine'
    epsilon = 0.1
    l2_lambda= 0.3
    learning_rate = 0.003
    max_grad_norm = 20.0
    keep_prob = 0.6
    hidden_layer_num = 1
    hidden_size = 150
    evaluation_interval = 5
    batch_size = 10
    epochs = 300
    allow_soft_placement = True
    log_device_placement = False
    save_test_logits = False
    train_data_path = 'data_fr_en/formatted_data_train_w.csv'
    test_data_path = 'data_fr_en/formatted_data_test_w.csv'

# for attr, value in sorted(FLAGS.__flags.items()):
#     print("{}={}".format(attr.upper(), value))
# print("")


In [ ]:
# The code is rewritten based on source code from tensorflow tutorial for Recurrent Neural Network.
# https://www.tensorflow.org/versions/0.6.0/tutorials/recurrent/index.html
# You can get source code for the tutorial from
# https://github.com/tensorflow/tensorflow/blob/master/tensorflow/models/rnn/ptb/ptb_word_lm.py
#
# There is dropout on each hidden layer to prevent the model from overfitting
#
# Here is an useful practical guide for training dropout networks
# https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf
# You can find the practical guide on Appendix A


def add_gradient_noise(t, stddev=1e-3, name=None):
    """
    Adds gradient noise as described in http://arxiv.org/abs/1511.06807 [2].

    The input Tensor `t` should be a gradient.

    The output will be `t` + gaussian noise.

    0.001 was said to be a good fixed value for memory networks [2].
    """
    with tf.name_scope(name, "add_gradient_noise", [t, stddev]) as name:
        t = tf.convert_to_tensor(t, name="t")
        gn = tf.random_normal(tf.shape(t), stddev=stddev)
        return tf.add(t, gn, name=name)


class StudentModel(object):
    def __init__(self, is_training, config):
        self._batch_size = batch_size = FLAGS.batch_size
        self.num_skills = num_skills = config.num_skills
        self.hidden_size = size = FLAGS.hidden_size
        self.num_steps = num_steps = config.num_steps
        input_size = num_skills * 2

        inputs = self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps])
        self._target_id = target_id = tf.placeholder(tf.int32, [None])
        self._target_correctness = target_correctness = tf.placeholder(tf.float32, [None])
        final_hidden_size = size
        hidden_layers = []
        for i in range(FLAGS.hidden_layer_num):
            final_hidden_size = size / (i + 1)
            hidden1 = tf.nn.rnn_cell.LSTMCell(final_hidden_size, state_is_tuple=True)
            if is_training and config.keep_prob < 1:
                hidden1 = tf.nn.rnn_cell.DropoutWrapper(hidden1, output_keep_prob=FLAGS.keep_prob)
            hidden_layers.append(hidden1)

        cell = tf.nn.rnn_cell.MultiRNNCell(hidden_layers, state_is_tuple=True)

        input_data = tf.reshape(self._input_data, [-1])
        # one-hot encoding
        with tf.device("/cpu:0"):
            labels = tf.expand_dims(input_data, 1)
            indices = tf.expand_dims(tf.range(0, batch_size * num_steps, 1), 1)
            concatenated = tf.concat([indices, labels], 1)
            inputs = tf.sparse_to_dense(concatenated, tf.stack([batch_size * num_steps, input_size]), 1.0, 0.0)
            inputs.set_shape([batch_size * num_steps, input_size])

        # [batch_size, num_steps, input_size]
        inputs = tf.reshape(inputs, [-1, num_steps, input_size])
        x = tf.transpose(inputs, [1, 0, 2])
        # Reshape to (n_steps*batch_size, n_input)
        x = tf.reshape(x, [-1, input_size])
        # Split to get a list of 'n_steps'
        # tensors of shape (doc_num, n_input)
        x = tf.split(x, num_steps, 0)
        # inputs = [tf.squeeze(input_, [1]) for input_ in tf.split(1, num_steps, inputs)]
        # outputs, state = tf.nn.rnn(hidden1, x, dtype=tf.float32)
        outputs, state = tf.contrib.rnn.static_rnn(cell, x, dtype=tf.float32)
        output = tf.reshape(tensor=tf.concat(outputs, 1), shape=[-1, tf.cast(final_hidden_size, dtype=tf.int32)])
        # calculate the logits from last hidden layer to output layer
        sigmoid_w = tf.get_variable("sigmoid_w", [final_hidden_size, num_skills])
        sigmoid_b = tf.get_variable("sigmoid_b", [num_skills])
        logits = tf.matmul(output, sigmoid_w) + sigmoid_b

        # from output nodes to pick up the right one we want
        logits = tf.reshape(logits, [-1])
        selected_logits = tf.gather(logits, self.target_id)
        self._all_logits = logits

        # make prediction
        self._pred = tf.sigmoid(selected_logits)

        # loss function
        loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=selected_logits, labels=target_correctness))

        # self._cost = cost = tf.reduce_mean(loss)
        self._cost = cost = loss

    @property
    def batch_size(self):
        return self._batch_size

    @property
    def input_data(self):
        return self._input_data

    @property
    def auc(self):
        return self._auc

    @property
    def pred(self):
        return self._pred

    @property
    def target_id(self):
        return self._target_id

    @property
    def target_correctness(self):
        return self._target_correctness

    @property
    def initial_state(self):
        return self._initial_state

    @property
    def all_logits(self):
        return self._all_logits

    @property
    def cost(self):
        return self._cost

    @property
    def final_state(self):
        return self._final_state


class HyperParamsConfig(object):
    """Small config."""
    init_scale = 0.05
    num_steps = 0
    max_grad_norm = FLAGS.max_grad_norm
    max_max_epoch = FLAGS.epochs
    keep_prob = FLAGS.keep_prob
    num_skills = 0


def run_epoch(session, m, students, eval_op, verbose=False):
    """Runs the model on the given data."""
    epoch_start_time = time.time()

    index = 0
    pred_labels = []
    actual_labels = []
    all_all_logits = []
    while index + m.batch_size < len(students):
        batch_start_time = time.time()
        x = np.zeros((m.batch_size, m.num_steps))
        target_id = []
        target_correctness = []
        count = 0
        for i in range(m.batch_size):
            student = students[index + i]
            problem_ids = student[1]
            correctness = student[2]
            for j in range(len(problem_ids) - 1):
                problem_id = int(problem_ids[j])
                label_index = 0
                if int(correctness[j]) == 0:
                    label_index = problem_id
                else:
                    label_index = problem_id + m.num_skills
                x[i, j] = label_index
                target_id.append(i * m.num_steps * m.num_skills + j * m.num_skills + int(problem_ids[j + 1]))
                target_correctness.append(int(correctness[j + 1]))
                actual_labels.append(int(correctness[j + 1]))

        index += m.batch_size

        pred, _, all_logits = session.run([m.pred, eval_op, m.all_logits], feed_dict={
            m.input_data: x, m.target_id: target_id,
            m.target_correctness: target_correctness})

        for p in pred:
            pred_labels.append(p)

        all_all_logits.append(all_logits)

        eta = ('Batch {} to {} ({:.3}%) completed in {:.4}s, epoch ETA: {:.4}s\t'
               .format(index - m.batch_size, index, index / len(students), time.time() - batch_start_time,
                       (time.time() - batch_start_time) * ((len(students) - index) / m.batch_size)))
        print('{}\r'.format(eta), end='', flush=True)

    rmse = sqrt(mean_squared_error(actual_labels, pred_labels))
    fpr, tpr, thresholds = metrics.roc_curve(actual_labels, pred_labels, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    # calculate r^2
    r2 = r2_score(actual_labels, pred_labels)
    f1 = f1_score(actual_labels, np.round(pred_labels))
    lloss = log_loss(actual_labels, pred_labels)
    acc = accuracy_score(actual_labels, np.round(pred_labels))
    print('\rTotal epoch time:', time.time() - epoch_start_time)
    return acc, lloss, f1, rmse, auc, r2, np.concatenate(all_all_logits)


def read_data_from_csv_file(fileName, shuffle=False):
    config = HyperParamsConfig()
    inputs = []
    targets = []
    rows = []
    max_skill_num = 0
    max_num_problems = 0
    with open(fileName, "r") as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            rows.append(row)
    index = 0
    i = 0
    print("the number of rows is " + str(len(rows)))
    tuple_rows = []
    # turn list to tuple
    while (index < len(rows) - 1):
        problems_num = int(rows[index][0])
        tmp_max_skill = max(map(int, rows[index + 1]))
        if (tmp_max_skill > max_skill_num):
            max_skill_num = tmp_max_skill
        if (problems_num <= 2):
            index += 3
        else:
            if problems_num > max_num_problems:
                max_num_problems = problems_num
            tup = (rows[index], rows[index + 1], rows[index + 2])
            tuple_rows.append(tup)
            index += 3
    # shuffle the tuple

    if shuffle:
        random.shuffle(tuple_rows)
    print("The number of students is ", len(tuple_rows))
    print("Finish reading data")
    return tuple_rows, max_num_problems, max_skill_num + 1


def main(unused_args):
    start_time = time.time()
    config = HyperParamsConfig()
    eval_config = HyperParamsConfig()
    timestamp = str(time.time())
    train_data_path = FLAGS.train_data_path
    # path to your test data set
    test_data_path = FLAGS.test_data_path
    # the file to store your test results
    result_file_path = "run_logs_{}".format(timestamp)
    # your model name
    model_name = "DKT"

    train_students, train_max_num_problems, train_max_skill_num = read_data_from_csv_file(train_data_path, shuffle=True)
    config.num_steps = train_max_num_problems
    print('train_max_num_problems=%d, train_max_skill_num=%d' % (train_max_num_problems, train_max_skill_num))

    test_students, test_max_num_problems, test_max_skill_num = read_data_from_csv_file(test_data_path, shuffle=False)
    config.num_skills = max(test_max_skill_num, train_max_skill_num)
    eval_config.num_steps = test_max_num_problems
    eval_config.num_skills = max(test_max_skill_num, train_max_skill_num)
    print('test_max_num_problems=%d, test_max_skill_num=%d' % (test_max_num_problems, test_max_skill_num))
    print('Reading time', round(time.time() - start_time, 3), 's')
    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(allow_soft_placement=FLAGS.allow_soft_placement,
                                      log_device_placement=FLAGS.log_device_placement)

        global_step = tf.Variable(0, name="global_step", trainable=False)
        # decay learning rate
        starter_learning_rate = FLAGS.learning_rate
        learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 3000, 0.96, staircase=True)

        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=FLAGS.epsilon)
        print('Time checkpoint', round(time.time() - start_time, 4), 's')
        print('Starting tf session...')
        with tf.Session(config=session_conf) as session:

            initializer = tf.random_uniform_initializer(-config.init_scale, config.init_scale)
            # training model
            with tf.variable_scope("model", reuse=None, initializer=initializer):
                m = StudentModel(is_training=True, config=config)
            # testing model
            with tf.variable_scope("model", reuse=True, initializer=initializer):
                mtest = StudentModel(is_training=False, config=eval_config)

            grads_and_vars = optimizer.compute_gradients(m.cost)
            grads_and_vars = [(tf.clip_by_norm(g, FLAGS.max_grad_norm), v)
                              for g, v in grads_and_vars if g is not None]
            grads_and_vars = [(add_gradient_noise(g), v) for g, v in grads_and_vars]
            train_op = optimizer.apply_gradients(grads_and_vars, name="train_op", global_step=global_step)
            print('Time checkpoint', round((time.time() - start_time) / 60, 4), 'min')
            print('Running session...')
            session.run(tf.global_variables_initializer())
            # log hyperparameters to results file
            with open(result_file_path, "a+") as f:
                print("Writing hyperparameters into file")
                f.write("Hidden layer size: %d \n" % (FLAGS.hidden_size))
                f.write("Dropout rate: %.3f \n" % (FLAGS.keep_prob))
                f.write("Batch size: %d \n" % (FLAGS.batch_size))
                f.write("Max grad norm: %d \n" % (FLAGS.max_grad_norm))
            saver = tf.train.Saver(tf.global_variables())
            print('Starting epochs...')
            print('Time checkpoint', round((time.time() - start_time) / 60, 4), 'min')
            for i in range(config.max_max_epoch):
                print('Running epoch {}...\r'.format(i + 1))
                acc, lloss, f1, rmse, auc, r2, _ = run_epoch(session, m, train_students, train_op, verbose=False)
                print("Epoch: %d Train Metrics:\n acc: %.3f log loss: %.3f \t f1: %.3f \t rmse: %.3f \t auc: %.3f \t r2: %.3f \n" 
                      % (i + 1, acc, lloss, f1, rmse, auc, r2))

                if (i + 1) % FLAGS.evaluation_interval == 0:
                    print("Save variables to disk")
                    save_path = saver.save(session, './' + model_name)
                    print("*" * 10)
                    print("Start to test model....")
                    acc, lloss, f1, rmse, auc, r2, all_logits = run_epoch(session, mtest, test_students, tf.no_op(), verbose=True)
                    print("Epoch: %d Test Metrics:\n acc: %.3f \t log loss: %.3f \t f1: %.3f \t rmse: %.3f \t auc: %.3f \t r2: %.3f" 
                          % (i + 1, acc, lloss, f1, rmse, auc, r2))
                    with open(result_file_path, "a+") as f:
                        f.write("Epoch: %d Test Metrics:\n rmse: %.3f \t auc: %.3f \t r2: %.3f" % (
                            (i + 1) / 2, rmse, auc, r2))
                        f.write("\n")

                        print("*" * 10)
                    # save the logits
                    if FLAGS.save_test_logits:
                        np.savetxt(result_file_path + ('.e%02d.logits' % (i + 1)), all_logits)


if __name__ == "__main__":
    tf.app.run()


the number of rows is 5010
The number of students is  1670
Finish reading data
train_max_num_problems=808, train_max_skill_num=3450
the number of rows is 3615
The number of students is  1205
Finish reading data
test_max_num_problems=1286, test_max_skill_num=3653
Reading time 0.4 s
Time checkpoint 0.4076 s
Starting tf session...
Time checkpoint 1.0548 min
Running session...
Writing hyperparameters into file
Starting epochs...
Time checkpoint 1.1229 min
Running epoch 1...
Total epoch time: 100.69543313980103d in 0.4304s, epoch ETA: 0.4304s	
Epoch: 1 Train Metrics:
 acc: 0.807 log loss: 0.453 	 f1: 0.275 	 rmse: 0.383 	 auc: 0.683 	 r2: -0.081 

Running epoch 2...
Total epoch time: 75.61328649520874ed in 0.4416s, epoch ETA: 0.4416s	
Epoch: 2 Train Metrics:
 acc: 0.848 log loss: 0.375 	 f1: 0.262 	 rmse: 0.341 	 auc: 0.766 	 r2: 0.147 

Running epoch 3...
Total epoch time: 75.56995892524719ed in 0.446s, epoch ETA: 0.4461s		
Epoch: 3 Train Metrics:
 acc: 0.850 log loss: 0.372 	 f1: 0.262 	 

Total epoch time: 75.34026551246643ed in 0.4314s, epoch ETA: 0.4314s	
Epoch: 33 Train Metrics:
 acc: 0.862 log loss: 0.336 	 f1: 0.375 	 rmse: 0.321 	 auc: 0.823 	 r2: 0.244 

Running epoch 34...
Total epoch time: 75.04298686981201ed in 0.4465s, epoch ETA: 0.4465s	
Epoch: 34 Train Metrics:
 acc: 0.863 log loss: 0.335 	 f1: 0.376 	 rmse: 0.320 	 auc: 0.824 	 r2: 0.246 

Running epoch 35...
Total epoch time: 75.08127212524414ed in 0.4162s, epoch ETA: 0.4162s	
Epoch: 35 Train Metrics:
 acc: 0.863 log loss: 0.335 	 f1: 0.379 	 rmse: 0.320 	 auc: 0.825 	 r2: 0.247 

Save variables to disk
**********
Start to test model....
Total epoch time: 48.22095608711243ed in 0.3551s, epoch ETA: 0.1776s	
Epoch: 35 Test Metrics:
 acc: 0.845 	 log loss: 0.373 	 f1: 0.342 	 rmse: 0.340 	 auc: 0.795 	 r2: 0.204
**********
Running epoch 36...
Total epoch time: 75.39515566825867ed in 0.4396s, epoch ETA: 0.4396s	
Epoch: 36 Train Metrics:
 acc: 0.863 log loss: 0.334 	 f1: 0.384 	 rmse: 0.320 	 auc: 0.825 	 r2: 

Total epoch time: 75.31276249885559ed in 0.4439s, epoch ETA: 0.444s		
Epoch: 66 Train Metrics:
 acc: 0.867 log loss: 0.325 	 f1: 0.417 	 rmse: 0.315 	 auc: 0.837 	 r2: 0.271 

Running epoch 67...
Total epoch time: 75.34398674964905ed in 0.4283s, epoch ETA: 0.4283s	
Epoch: 67 Train Metrics:
 acc: 0.867 log loss: 0.325 	 f1: 0.415 	 rmse: 0.315 	 auc: 0.836 	 r2: 0.270 

Running epoch 68...
Total epoch time: 75.20028614997864ed in 0.4336s, epoch ETA: 0.4336s	
Epoch: 68 Train Metrics:
 acc: 0.867 log loss: 0.325 	 f1: 0.417 	 rmse: 0.315 	 auc: 0.837 	 r2: 0.272 

Running epoch 69...


/home/sjsarsa/duolingo_dkt/code/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1694: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/sjsarsa/duolingo_dkt/code/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1694: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


Total epoch time: 75.32687973976135
Epoch: 69 Train Metrics:
 acc: 0.867 log loss: nan 	 f1: 0.418 	 rmse: 0.314 	 auc: 0.837 	 r2: 0.273 

Running epoch 70...
Total epoch time: 75.16026902198792ed in 0.4297s, epoch ETA: 0.4297s	
Epoch: 70 Train Metrics:
 acc: 0.867 log loss: 0.324 	 f1: 0.420 	 rmse: 0.314 	 auc: 0.838 	 r2: 0.274 

Save variables to disk
**********
Start to test model....
Total epoch time: 48.24901223182678ed in 0.355s, epoch ETA: 0.1775s		
Epoch: 70 Test Metrics:
 acc: 0.848 	 log loss: 0.367 	 f1: 0.381 	 rmse: 0.337 	 auc: 0.803 	 r2: 0.221
**********
Running epoch 71...
Total epoch time: 75.49891328811646ed in 0.4389s, epoch ETA: 0.4389s	
Epoch: 71 Train Metrics:
 acc: 0.867 log loss: 0.324 	 f1: 0.419 	 rmse: 0.314 	 auc: 0.838 	 r2: 0.274 

Running epoch 72...
Total epoch time: 75.24097776412964ed in 0.4428s, epoch ETA: 0.4428s	
Epoch: 72 Train Metrics:
 acc: 0.868 log loss: 0.324 	 f1: 0.420 	 rmse: 0.314 	 auc: 0.838 	 r2: 0.274 

Running epoch 73...
Total ep

...

Total epoch time: 48.35918951034546ed in 0.3592s, epoch ETA: 0.1796s	
Epoch: 135 Test Metrics:
 acc: 0.850 	 log loss: 0.364 	 f1: 0.404 	 rmse: 0.335 	 auc: 0.807 	 r2: 0.230
**********
Running epoch 136...
Total epoch time: 75.04819130897522ed in 0.427s, epoch ETA: 0.427s	s	
Epoch: 136 Train Metrics:
 acc: 0.873 log loss: nan 	 f1: 0.462 	 rmse: 0.308 	 auc: 0.852 	 r2: 0.303 

Running epoch 137...
Total epoch time: 75.40665602684021ed in 0.4456s, epoch ETA: 0.4456s	
Epoch: 137 Train Metrics:
 acc: 0.873 log loss: nan 	 f1: 0.459 	 rmse: 0.308 	 auc: 0.852 	 r2: 0.302 

Running epoch 138...
Total epoch time: 75.14399361610413ed in 0.4346s, epoch ETA: 0.4346s	
Epoch: 138 Train Metrics:
 acc: 0.872 log loss: nan 	 f1: 0.460 	 rmse: 0.308 	 auc: 0.853 	 r2: 0.303 

Running epoch 139...
Total epoch time: 75.21467018127441ed in 0.4404s, epoch ETA: 0.4404s	
Epoch: 139 Train Metrics:
 acc: 0.873 log loss: nan 	 f1: 0.462 	 rmse: 0.308 	 auc: 0.852 	 r2: 0.303 

Running epoch 140...
Total ep

Total epoch time: 75.47207307815552ed in 0.4319s, epoch ETA: 0.4319s	
Epoch: 169 Train Metrics:
 acc: 0.875 log loss: nan 	 f1: 0.475 	 rmse: 0.305 	 auc: 0.857 	 r2: 0.314 

Running epoch 170...
Total epoch time: 74.983891248703eted in 0.4325s, epoch ETA: 0.4325s	
Epoch: 170 Train Metrics:
 acc: 0.875 log loss: nan 	 f1: 0.478 	 rmse: 0.305 	 auc: 0.858 	 r2: 0.315 

Save variables to disk
**********
Start to test model....
Total epoch time: 47.911972522735596d in 0.352s, epoch ETA: 0.176s	s	
Epoch: 170 Test Metrics:
 acc: 0.851 	 log loss: 0.365 	 f1: 0.389 	 rmse: 0.334 	 auc: 0.808 	 r2: 0.231
**********
Running epoch 171...
Total epoch time: 75.18378973007202ed in 0.4256s, epoch ETA: 0.4256s	
Epoch: 171 Train Metrics:
 acc: 0.875 log loss: nan 	 f1: 0.476 	 rmse: 0.306 	 auc: 0.857 	 r2: 0.313 

Running epoch 172...
Total epoch time: 75.28766798973083ed in 0.4397s, epoch ETA: 0.4397s	
Epoch: 172 Train Metrics:
 acc: 0.875 log loss: nan 	 f1: 0.476 	 rmse: 0.305 	 auc: 0.857 	 r2: 

Total epoch time: 75.21609902381897ed in 0.4409s, epoch ETA: 0.4409s	
Epoch: 202 Train Metrics:
 acc: 0.876 log loss: nan 	 f1: 0.485 	 rmse: 0.304 	 auc: 0.861 	 r2: 0.322 

Running epoch 203...
Total epoch time: 75.45384645462036ed in 0.4357s, epoch ETA: 0.4357s	
Epoch: 203 Train Metrics:
 acc: 0.876 log loss: nan 	 f1: 0.487 	 rmse: 0.304 	 auc: 0.861 	 r2: 0.322 

Running epoch 204...
Total epoch time: 75.34532856941223ed in 0.4383s, epoch ETA: 0.4383s	
Epoch: 204 Train Metrics:
 acc: 0.877 log loss: nan 	 f1: 0.489 	 rmse: 0.303 	 auc: 0.862 	 r2: 0.325 

Running epoch 205...
Total epoch time: 75.20156145095825ed in 0.4386s, epoch ETA: 0.4386s	
Epoch: 205 Train Metrics:
 acc: 0.876 log loss: nan 	 f1: 0.488 	 rmse: 0.303 	 auc: 0.862 	 r2: 0.324 

Save variables to disk
**********
Start to test model....
Total epoch time: 48.026294469833374d in 0.347s, epoch ETA: 0.1735s		
Epoch: 205 Test Metrics:
 acc: 0.850 	 log loss: 0.366 	 f1: 0.406 	 rmse: 0.335 	 auc: 0.805 	 r2: 0.227
***

Total epoch time: 48.147268533706665d in 0.3463s, epoch ETA: 0.1731s	
Epoch: 235 Test Metrics:
 acc: 0.849 	 log loss: 0.369 	 f1: 0.403 	 rmse: 0.336 	 auc: 0.803 	 r2: 0.222
**********
Running epoch 236...
Total epoch time: 74.77028703689575ed in 0.4259s, epoch ETA: 0.4259s	
Epoch: 236 Train Metrics:
 acc: 0.878 log loss: nan 	 f1: 0.498 	 rmse: 0.301 	 auc: 0.866 	 r2: 0.334 

Running epoch 237...
Total epoch time: 75.45186066627502ed in 0.4487s, epoch ETA: 0.4487s	
Epoch: 237 Train Metrics:
 acc: 0.878 log loss: nan 	 f1: 0.498 	 rmse: 0.301 	 auc: 0.865 	 r2: 0.332 

Running epoch 238...
Total epoch time: 75.37628173828125ed in 0.4431s, epoch ETA: 0.4431s	
Epoch: 238 Train Metrics:
 acc: 0.878 log loss: nan 	 f1: 0.499 	 rmse: 0.301 	 auc: 0.866 	 r2: 0.332 

Running epoch 239...
Total epoch time: 75.46387887001038ed in 0.4359s, epoch ETA: 0.4359s	
Epoch: 239 Train Metrics:
 acc: 0.878 log loss: nan 	 f1: 0.500 	 rmse: 0.301 	 auc: 0.867 	 r2: 0.335 

Running epoch 240...
Total ep

Total epoch time: 76.70652985572815ed in 0.4447s, epoch ETA: 0.4447s	
Epoch: 269 Train Metrics:
 acc: 0.879 log loss: nan 	 f1: 0.508 	 rmse: 0.299 	 auc: 0.869 	 r2: 0.341 

Running epoch 270...
Total epoch time: 76.04609370231628ed in 0.4245s, epoch ETA: 0.4245s	
Epoch: 270 Train Metrics:
 acc: 0.879 log loss: nan 	 f1: 0.506 	 rmse: 0.300 	 auc: 0.869 	 r2: 0.340 

Save variables to disk
**********
Start to test model....
Total epoch time: 48.854592084884644d in 0.3617s, epoch ETA: 0.1809s	
Epoch: 270 Test Metrics:
 acc: 0.848 	 log loss: nan 	 f1: 0.407 	 rmse: 0.337 	 auc: 0.801 	 r2: 0.217
**********
Running epoch 271...
Total epoch time: 76.01705241203308ed in 0.4292s, epoch ETA: 0.4293s	
Epoch: 271 Train Metrics:
 acc: 0.879 log loss: nan 	 f1: 0.505 	 rmse: 0.300 	 auc: 0.869 	 r2: 0.338 

Running epoch 272...
Total epoch time: 75.8475387096405ted in 0.424s, epoch ETA: 0.424s	s	
Epoch: 272 Train Metrics:
 acc: 0.879 log loss: nan 	 f1: 0.508 	 rmse: 0.300 	 auc: 0.869 	 r2: 0.